In [ ]:
# Create TEST DF for testing new functionalities

# Read a parquet file and create a new sample (50) and save it as a parquet file

# test_df = pd.read_parquet('../../../../data/data_processed/us_video_ads_batch/us_video_ads_sample_1.parquet')
# # test_df.head()

# test_scrape_df = test_df.sample(40)
# test_scrape_df.to_parquet('../../../../data/data_processed/test/test_us_video_ads_sample.parquet', compression='gzip')
# test_scrape_df.shape

In [ ]:
num_iterations = 5
range(num_iterations)

In [1]:
# Check for matching rows in parquet and csv files

import glob
import pandas as pd

test_df = pd.read_parquet('../../../../data/data_processed/us_video_ads_batch/us_video_ads_sample_145.parquet')
test_ad_ids = set(test_df['ad_id'])

files = glob.glob('../../../../data/adspider/scrapy_us_video_batch_*.csv')
matching_files = []
matching_rows_count = 0
for file in files:
    try:
        df = pd.read_csv(file)
        matching_rows = set(df['ad_id']).intersection(test_ad_ids)
        if matching_rows:
            matching_files.append(file)
            matching_rows_count += len(matching_rows)
    except pd.errors.EmptyDataError:
        # print(f"Empty file: {file}")
        pass

matching_files.sort()

print(f"Matching files: {matching_files}")
print(f"Matching rows count: {matching_rows_count}")
# print(test_df.head())



Matching files: ['../../../../data/adspider/scrapy_us_video_batch_20230527T005337_12.csv', '../../../../data/adspider/scrapy_us_video_batch_20230527T005337_13.csv', '../../../../data/adspider/scrapy_us_video_batch_20230527T005337_14.csv', '../../../../data/adspider/scrapy_us_video_batch_20230527T005337_15.csv', '../../../../data/adspider/scrapy_us_video_batch_20230527T005337_16.csv']
Matching rows count: 2048


In [3]:
# Test functions for scraping single file

import os
import random
import pickle

folder = "../../../../data/data_processed/us_video_ads_batch"
with open("../../../processed_files.pkl", "rb") as f:
    processed_files = pickle.load(f)


total_unprocessed_files = [
    filename
    for filename in os.listdir(folder)
    if filename.endswith(".parquet")
    and filename not in processed_files
]
print(total_unprocessed_files)

max_unprocessed_files = 3
unprocessed_files = random.sample(total_unprocessed_files, max_unprocessed_files)

print(unprocessed_files)
print(total_unprocessed_files[:max_unprocessed_files])




['us_video_ads_sample_94.parquet', 'us_video_ads_sample_113.parquet', 'us_video_ads_sample_138.parquet', 'us_video_ads_sample_73.parquet', 'us_video_ads_sample_147.parquet', 'us_video_ads_sample_120.parquet', 'us_video_ads_sample_100.parquet', 'us_video_ads_sample_6.parquet', 'us_video_ads_sample_146.parquet', 'us_video_ads_sample_90.parquet', 'us_video_ads_sample_107.parquet', 'us_video_ads_sample_85.parquet', 'us_video_ads_sample_45.parquet', 'us_video_ads_sample_150.parquet', 'us_video_ads_sample_108.parquet', 'us_video_ads_sample_28.parquet', 'us_video_ads_sample_21.parquet', 'us_video_ads_sample_106.parquet', 'us_video_ads_sample_74.parquet', 'us_video_ads_sample_3.parquet', 'us_video_ads_sample_133.parquet', 'us_video_ads_sample_141.parquet', 'us_video_ads_sample_142.parquet', 'us_video_ads_sample_18.parquet', 'us_video_ads_sample_56.parquet', 'us_video_ads_sample_91.parquet', 'us_video_ads_sample_82.parquet', 'us_video_ads_sample_119.parquet', 'us_video_ads_sample_32.parquet', '

In [3]:
import glob
import os
import pandas as pd

folder_path = '../../../../data/data_processed/us_video_ads_batch/*.parquet'
all_us_video = pd.concat([pd.read_parquet(f) for f in glob.glob(folder_path)])

folder_path = '../../../../data/adspider/*.csv'
dfs = []
for f in glob.glob(folder_path):
    try:
        df = pd.read_csv(f)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        # Delete empty file
        print(f"Deleting empty file: {f}")
        os.remove(f)

scraped_videos = pd.concat(dfs)
print(scraped_videos.shape)
print(scraped_videos['ad_id'].nunique())




Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230525T162250_17.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230521T015421_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230520T020357_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230520T162424_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230521T140036_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230520T173259_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230521T002532_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230523T143131_17.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230523T131857_17.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230520T184241_9.csv
Deleting empty file: ../../../../data/adspider/scrapy_us_video_batch_20230520T025232_9.

In [8]:
# Save a list of rows that erronous didn't get scraped.

# all_us_video and scraped_videos are your two DataFrames
ad_id_count_df1 = all_us_video['ad_id'].nunique()
ad_id_count_df2 = scraped_videos['ad_id'].nunique()

difference = ad_id_count_df1 - ad_id_count_df2
print(f"DataFrame 1 has {difference} more unique ad_id values than Scraped Dataframe 2")

# Find the rows in all_us_video that don't coincide with scraped_videos
all_us_video_unique_rows = all_us_video[~all_us_video['ad_id'].isin(scraped_videos['ad_id'])]

# Save the unique rows to a parquet file
all_us_video_unique_rows.to_parquet('../../../../data/data_processed/us_video_ads_batch/us_video_ads_sample_155_final.parquet')


DataFrame 1 has 12 more unique ad_id values than Scraped Dataframe 2
